# Assigment 1, $\color{red}{\text{Step 1}}$
Please discard the whole transmission network, resulting in a cupper‐plate system. This means that all demands and all generation units are located in a single node. This assumption will be relaxed in future steps. Please also discard the information about reserve capacity of generators (it will be needed in Step 4).
1. Assuming all 17 demands are inelastic to price, solve the market‐clearing optimization problem in Julia, and report market‐clearing outcomes, including the total production cost of the system, production quantity of each generator, and the market‐clearing price (under uniform pricing). In addition, please compute the profit (not revenue) of each conventional generator and that of each wind farm if the underlying pricing scheme is (a) uniform, (b) pay‐as‐bid, i.e., every one is paid based on what she bided.

2. Now, let’s assume all demands are elastic to price. Please give arbitrarily selected values for their bid prices. Solve the market‐clearing optimization problem in Julia, and report market‐clearing outcomes. Ideally, tune bid price of demands in a way that a part of the whole demand (around 10%) will not be supplied. Please report the social welfare, production quantity of each generator, consumption quantity of each demand, the market‐clearing price (under uniform pricing), the profit (not revenue) of each conventional generator and each wind farm as well as the utility (not payment) of each demand if the underlying pricing scheme is (a) uniform, (b) pay‐as‐bid.

2. The bid price of demands and the offer price of generators/farms are private information (these information are only available to the market operator). Assuming these information are public, discuss how each generator/farm/demand will change her offer/bid price to maximize her own profit/utility if the underlying pricing scheme is (a) uniform, (b) pay‐as‐bid.

In [96]:
using Pkg
Pkg.add("NBInclude");

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [97]:
using NBInclude
@nbinclude("Assigment1_Input_data.ipynb");

## 1 Market clearing optimization problem assuming demands are inelastic to price
Assuming all 17 demands are inelastic to price, solve the market‐clearing optimization problem in Julia, and report market‐clearing outcomes, including the total production cost of the system, production quantity of each generator, and the market‐clearing price (under uniform pricing). In addition, please compute the profit (not revenue) of each conventional generator and that of each wind farm if the underlying pricing scheme is (a) uniform, (b) pay‐as‐bid, i.e., every one is paid based on what she bided.

\begin{aligned}
\min_{p^{G}_g,p^{W}_w} \quad & \sum_{g=1}^{N_{G}}{C_{g}p^{G}_g} + \sum_{w=1}^{N_{W}}{T_{w} p^{W}_w}\\
\textrm{s.t.} \quad & 0 \geq p^{D}_{d} \geq \bar{P^{D}_{d}} \quad \quad \forall d\\
  & 0 \geq p^{G}_{g} \geq \bar{P^{G}_{g}} \quad \quad \forall g \\
  & 0 \geq p^{W}_{w} \geq \bar{P^{W}_{w}} \quad \quad \forall w \\
  & P^D - \sum_{g=1}^{N_{G}}{p^{G}_g} - \sum_{w=1}^{N_{W}}{p^{W}_w} = 0 \quad :\lambda \\
\end{aligned}

$U_{d}:$ bid price of demand $d$ $[€/MWh]$

$C_{g}:$ offer price of conventional generators $g$ $[€/MWh]$

$T_{w}:$ offer price of wind turbines $w$ $[€/MWh]$

#### Matrix model

$U:$ row vector of bid price of demand $(N_{d} x 1)$

$C:$ row vector of offer price of generators $(N_{g} x 1)$

$T:$ row vector of offer price of wind turbines $(N_{w} x 1)$



P_gen_max: column vector of maximum capacity of conventional generators $(1 x N_{g})$

P_win_max: column vector of maximum generation of wind turbines $(1 x N_{w})$

In [98]:
## 1.1 Input
# variable constraints parameters
P̅ᴳ = cg_capacity
P̲ᴳ = zeros(N_G)
P̅ᵂ = wf_day_ahead_forecast
P̲ᵂ = zeros(N_W)

# cost parameters
C = (cg_capacity/2)'
T = zeros(4)'

# inelastic demands parameters
pᴰ= Demands[2,:]; # Demends inelastic price

In [99]:
## formulation
using JuMP
using GLPK

inelastic_model = Model(GLPK.Optimizer)

@variable(inelastic_model, pᴳ[1:N_G])
@variable(inelastic_model, pᵂ[1:N_W])

@constraint(inelastic_model, cg_con[i=1:N_G], P̲ᴳ[i] <= pᴳ[i] <= P̅ᴳ[i])
@constraint(inelastic_model, wt_con[i=1:N_W], P̲ᵂ[i] <= pᵂ[i] <= P̅ᵂ[i])
@constraint(inelastic_model, balance_con, + sum(pᴰ) - sum(pᴳ) - sum(pᵂ) .== 0)

@objective(inelastic_model, Min, T*pᵂ + C*pᴳ)

optimize!(inelastic_model)

@show solve_time(inelastic_model)
@show objective_value(inelastic_model)
@show value.(pᴳ)
@show value.(pᵂ)
@show shadow_price(balance_con);

solve_time(inelastic_model) = 4.792213439941406e-5
objective_value(inelastic_model) = 198284.31
value.(pᴳ) = [106.4, 106.4, 245.0, 0.0, 42.0, 108.5, 108.5, 210.6400000000001, 280.0, 210.0, 217.0, 245.0]
value.(pᵂ) = [120.54, 115.52, 53.34, 38.16]
shadow_price(balance_con) = -140.0


In [100]:
## Results
λ_𝒰 = abs(shadow_price(balance_con)) # Market Clearing Price

pᴳ_𝒰 = value.(pᴳ) # p\scrD\scrA\^G
pᵂ_𝒰 = value.(pᵂ)
pᴰ_𝒰 = value.(pᴰ);

In [101]:
## Profit computation

### Under uniform pricing
Rᴳ_𝒰_uni = pᴳ_𝒰 * λ_𝒰 # Revenue of conventional generators
cost_fuel_𝒰 = cg_production_cost .* pᴳ_𝒰
Profitᴳ_𝒰_uni = Rᴳ_𝒰_uni - cost_fuel_𝒰

Rᵂ_𝒰_uni = pᵂ_𝒰 * λ_𝒰; # Revenue of wind farms
Profitᵂ_𝒰_uni = Rᵂ_𝒰_uni #Profit

### Under pay-as-bid pricing
Rᴳ_𝒰_pab = C' .* pᴳ_𝒰 
Profitᴳ_𝒰_pab = (C' - cg_production_cost) .* pᴳ_𝒰; 

Rᵂ_𝒰_pab =  T' .* pᵂ_𝒰
Profitᵂ_𝒰_pab = Rᵂ_𝒰_pab;

## 2 Market clearing optimization problem assuming demands are elastic to price
Now, let’s assume all demands are elastic to price. Please give arbitrarily selected values for their bid prices. Solve the market‐clearing optimization problem in Julia, and report market‐clearing outcomes. Ideally, tune bid price of demands in a way that a part of the whole demand (around 10%) will not be supplied. Please report the social welfare, production quantity of each generator, consumption quantity of each demand, the market‐clearing price (under uniform pricing), the profit (not revenue) of each conventional generator and each wind farm as well as the utility (not payment) of each demand if the underlying pricing scheme is (a) uniform, (b) pay‐as‐bid.


#### Compact model

\begin{aligned}
\max_{p^{G}_g,p^{W}_w,p^{D}_d} \quad & SW = \sum_{d=1}^{N_{D}}{U_{d} p^{D}_d} - \sum_{g=1}^{N_{G}}{C_{g}p^{G}_g} - \sum_{w=1}^{N_{W}}{T_{w} p^{W}_w}\\
\textrm{s.t.} \quad & 0 \geq p^{D}_{d} \geq \bar{P^{D}_{d}} \quad \quad \forall d\\
  & 0 \geq p^{G}_{g} \geq \bar{P^{G}_{g}} \quad \quad \forall g \\
  & 0 \geq p^{W}_{w} \geq \bar{P^{W}_{w}} \quad \quad \forall w \\
  & \sum_{d=1}^{N_{D}}{p^{D}_d} - \sum_{g=1}^{N_{G}}{p^{G}_g} - \sum_{w=1}^{N_{W}}{p^{W}_w} = 0 \quad :\lambda \\
\end{aligned}

$U_{d}:$ bid price of demand $d$ $[€/MWh]$

$C_{g}:$ offer price of conventional generators $g$ $[€/MWh]$

$T_{w}:$ offer price of wind turbines $w$ $[€/MWh]$

#### Matrix model

$U:$ row vector of bid price of demand $(N_{d} x 1)$

$C:$ row vector of offer price of generators $(N_{g} x 1)$

$T:$ row vector of offer price of wind turbines $(N_{w} x 1)$

P_dem_max: column vector of maximum load of demands $(1 x N_{d})$

P_gen_max: column vector of maximum capacity of conventional generators $(1 x N_{g})$

P_win_max: column vector of maximum generation of wind turbines (wind turbines forecasting) $(1 x N_{w})$

In [102]:
## 1.1 Input
# variable constraints parameters
P̅ᴳ = cg_capacity
P̲ᴳ = zeros(N_G)
P̅ᵂ = wf_day_ahead_forecast
P̲ᵂ = zeros(N_W)

# cost parameters
C = (cg_capacity/2)'
T = zeros(4)'
U = (Demands[2,:] + ones(N_D) * 60)'

# inelastic demands parameters
P̅ᴰ = Demands[2,:] # Demends elastic price
P̲ᴰ = zeros(N_D);

In [103]:
## formulation
using JuMP
using GLPK

elastic_model = Model(GLPK.Optimizer)

@variable(elastic_model, pᴳ[1:N_G])
@variable(elastic_model, pᵂ[1:N_W])
@variable(elastic_model, pᴰ[1:N_D])


@constraint(elastic_model, cg_con[i=1:N_G], P̲ᴳ[i] <= pᴳ[i] <= P̅ᴳ[i])
@constraint(elastic_model, wt_con[i=1:N_W], P̲ᵂ[i] <= pᵂ[i] <= P̅ᵂ[i])
@constraint(elastic_model, dem_con[i=1:N_D], P̲ᴰ[i] <= pᴰ[i] <= P̅ᴰ[i])

@constraint(elastic_model, balance_con, + sum(pᴰ) - sum(pᴳ) - sum(pᵂ) .== 0)

@objective(elastic_model, Max, U*pᴰ - T*pᵂ - C*pᴳ)

optimize!(elastic_model)

@show solve_time(elastic_model)
@show objective_value(elastic_model)
@show value.(pᴳ)
@show value.(pᵂ)
@show value.(pᴰ)
@show shadow_price(balance_con);

solve_time(elastic_model) = 0.00010895729064941406
objective_value(elastic_model) = 281501.69
value.(pᴳ) = [106.4, 106.4, 245.0, 0.0, 42.0, 108.5, 108.5, 225.64, 0.0, 210.0, 217.0, 245.0]
value.(pᵂ) = [120.54, 115.52, 53.34, 38.16]
value.(pᴰ) = [84.0, 0.0, 139.0, 0.0, 0.0, 106.0, 97.0, 132.0, 135.0, 150.0, 205.0, 150.0, 245.0, 0.0, 258.0, 141.0, 100.0]
shadow_price(balance_con) = 140.0


In [104]:
## Results
λ_ℰ = shadow_price(balance_con)
SW = objective_value(elastic_model)

pᴳ_ℰ = value.(pᴳ) # p\scrD\scrA\^G
pᵂ_ℰ = value.(pᵂ)
pᴰ_ℰ = value.(pᴰ);
println(sum(P̅ᴰ))
println(sum(pᴰ_ℰ))

2207
1942.0


In [105]:
## Profit computation

### Under uniform pricing
Rᴳ_ℰ_uni = pᴳ_ℰ * λ_ℰ # Revenue of conventional generators
cost_fuel_ℰ = cg_production_cost .* pᴳ_ℰ
Profitᴳ_ℰ_uni = Rᴳ_ℰ_uni - cost_fuel_ℰ # Profit of each conventional generator under (a) uniform pricing scheme

Rᵂ_ℰ_uni = pᵂ_ℰ * λ_ℰ; # Revenue of wind farms
Profitᵂ_ℰ_uni = Rᵂ_ℰ_uni # Profit of each wind farm under (a) uniform pricing scheme

Demands_Payment = λ_ℰ * pᴰ_ℰ
Curtaliment_cost = 500 * (P̅ᴰ - pᴰ_ℰ)
Utility_cost = Demands_Payment - Curtaliment_cost # Utility of each demand under (a) uniform pricing scheme

### Under pay-as-bid pricing
Rᴳ_ℰ_pab = C' .* pᴳ_ℰ 
Profitᴳ_ℰ_pab = (C' - cg_production_cost) .* pᴳ_ℰ; # Profit of each conventional generator under (b) pay-as-bid pricing scheme

Rᵂ_ℰ_pab =  T' .* pᵂ_ℰ
Profitᵂ_ℰ_pab = Rᵂ_ℰ_pab;

Demands_Payment_pab = U' .* pᴰ_ℰ
Utility_cost_pab = Demands_Payment_pab - Curtaliment_cost; # Utility of each demand under (b) pay-as-bid pricing scheme